<a href="https://colab.research.google.com/github/24-FYP-Automated-Feedback-Generation/Dataset_creation/blob/main/Another_copy_of_metacognitive_feedback_annotation_gpt_4o_mini_and_mistral.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Package installation

In [1]:
pip install mistralai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.3/261.3 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.2 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1


In [2]:
pip install python-dotenv

In [25]:
os.environ["MISTRAL_API_KEY"] = "Qmo2CawcS9Ee0WkHOGUNzB4epsKmQel2"

In [26]:
import os
from dotenv import load_dotenv


# Load environment variables from the .env file
load_dotenv()

# Access the API key using os.getenv
api_key = os.getenv("MISTRAL_API_KEY")

In [5]:
import asyncio
import os
from mistralai import Mistral, UserMessage

In [27]:
client = Mistral(api_key=api_key)
model = "mistral-large-latest"
from transformers import MistralModel, AutoTokenizer

In [7]:
# import torch

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# OpenAI Client

# Prompt to generate Metacognitive feedback

In [28]:
def generate_response(prompt, model="mistral-large-latest"):
    # Assuming 'client' is an instance of the relevant API client, it should be initialized prior to this function
    messages = [
        {
            "role": "system",
            "content": "You are a tutor providing metacognitive feedback to students working on programming problems. \
                        You need to give feedback for their algorithm design question answers. \
                        You should personalize this feedback to their metacognition skill level. \
                        But you need to focus more on identifying the errors and optimization in the students' code and guide them to problem solving without revealing the direct answer. \
                        Do not provide direct code answer for the problem."
        },
        {
            "role": "user",
            "content": prompt
        }
    ]

    chat_response = client.chat.complete(
        model=model,
        messages=messages,
    )

    return chat_response.choices[0].message.content


In [29]:
from langchain.prompts import PromptTemplate

template = """
You are a tutor providing metacognitive feedback to students working on programming problems. \
You need to give feedback for their algorithm design question answers. \
You should personalize this feedback to their metacognition skill level. \
But you need to focus more on identifying the errors and optimization in the students code and guide them to problem solving without revealing the direct answer. \
Do not provide direct code answer for the problem.
You should provide metacognitive feedback to students working on programming problems. The student has answered a programming question, and they have a metacognitive profile.

*Tree of Thoughts Steps**:

1. **Analyze the Student’s Answer**: Evaluate if the student’s answer correctly addresses the problem requirements. Identify any errors or key areas needing improvement.

   Problem: {problem}
   Student's Answer: {answer}
   Expected Correct Solution: {solution}

2. **Compare to Metacognitive Profile**:

    **Metacognitive Questions**:
    1. I read the question entirely, before I start the solving process.
    2. I identify and highlight the key requirements, inputs, outputs, and constraints of the programming task.
    3. I rephrase/summarize the question in my own words and identify the main points.
    4. I create specific input examples and manually work through them to reach the outputs to understand the problem better before thinking about the algorithm.
    5. I break down the problem statement into smaller, achievable sub-goals before beginning the implementation.
    6. Before I solve the problem, I estimate/think about the nature of the possible algorithm by recognizing patterns such as repetition and conditional that I would get.
    7. I sketch out the algorithm or plan the solution before start coding.
    8. Every time I revise and execute the designed algorithm systematically to reach the answer.
    9. I am always vigilant on the implementation process to verify that I am on the correct way to the solution.
    10. I pay attention to avoid negligent mistakes during the implementation process.
    11. I keep an eye on the problem-solving steps, which helps me to verify intermediate results.
    12. I always monitor the ongoing program implementation process.
    13. I check if the algorithm is acceptable and compatible with given data constraints.
    14. I confirm that the final implementation is correct.
    15. I refer again to the problem statement and check if the implemented solution is acceptable for all the given problem requirements.
    16. I refer to similar problems solved earlier and reflect on the accuracy and efficiency of my code solution.

    The student's responses to these questions are recorded as a 16-dimensional vector with values of {{1, 2, 3}} corresponding to {{almost never, sometimes, often}}. Use this vector to understand the student's self-reported strengths and areas where they may need guidance.
    Metacognitive Profile (16-Dimension Vector): {metacognitive_vector}
    Use the metacognitive profile to understand the student's correct metacognitive skill level. Identify the skills the student need to improve.

3. **Generate Metacognitive Feedback**: Generate feedback that addresses errors and improvements in the student's code (step 1). \
Focus primarily on the code, but if issues stem from weak metacognitive skills, guide the student to resolve the problem and enhance those skills simultaneously.\
Do not repeat unrelated meatacognition skills. If the identified issue/improvement is not aligned with the skill, then do not mention it in the feedback. \
Only focus on the skills that requires to solve the issues in the student's code.

Final output should address the following:
   -DO NOT include code solutions
   -Only output the final metacognitive feedback generated under step 3.
   -Should be less than 500 words
   -Do not mention about the questionnnaire.
   -NO NOT REPEAT THE QUESTIONS IN THE STEP 2 ABOVE.
   -Focus more on giving specific hints to solve the problem, while improving metacognitve skills.
### Metacognitive Feedback:
guide how to solve the problem and correct mistakes.Output should be a single paragraph less than 400 words. Output only the metacognitive feedback. Feedback should not repeat the metacogntion questions.
"""
prompt_template = PromptTemplate(
    input_variables=["problem", "answer", "metacognitive_vector", "solution"],
    template=template
)

# Data Preprocessing

In [10]:
import pandas as pd

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
file_path = "/content/drive/MyDrive/final_dataset_with_annotated_metacognitive_feedback_gpt-4o-mini.csv"
df= pd.read_csv(file_path)

In [13]:
df.shape

(16806, 21)

In [14]:
df.columns

Index(['Question 1', 'Response 1', 'Right answer 1', 'Q01', 'Q02', 'Q03',
       'Q04', 'Q05', 'Q06', 'Q07', 'Q08', 'Q09', 'Q10', 'Q11', 'Q12', 'Q13',
       'Q14', 'Q15', 'Q16', 'metacognitive_vector', 'metacognitive_feedback'],
      dtype='object')

In [15]:
df.rename(
    columns={
        'Question 1': 'Problem',
        'Response 1': 'Student_code',
        'Right answer 1': 'Expected_code'
    },
    inplace=True
)

In [ ]:
for col in df.columns:
    if col.startswith('Q'):  # Check if the column is one of the Q columns
        df[col] = df[col].apply(lambda x: x.split(':')[0])

In [ ]:
metacognitive_vectors = []  # Create a list to store vectors for each row

for _, row in df.iterrows():
    # Extract the values for columns starting with 'Q'
    row_values = [row[col] for col in df.columns if col.startswith('Q')]
    # Append the string representation of the vector for this row
    metacognitive_vectors.append(str(row_values))

# Assign the list of vectors to the new column after the loop
df['metacognitive_vector'] = metacognitive_vectors


In [ ]:
df.head(4)

,Problem,Student_code,Expected_code,Q01,Q02,Q03,Q04,Q05,Q06,Q07,...,Q09,Q10,Q11,Q12,Q13,Q14,Q15,Q16,metacognitive_vector,metacognitive_feedback
0,Develop a Python program that takes the name o...,file_input = input() file_open = open(fil...,"def substitute_vowels(chunk, vowel_substitutes...",3 : Often,2 : Sometimes,1 : Almost Never,3 : Often,2 : Sometimes,2 : Sometimes,1 : Almost Never,...,3 : Often,3 : Often,2 : Sometimes,2 : Sometimes,2 : Sometimes,3 : Often,3 : Often,1 : Almost Never,"['3 ', '2 ', '1 ', '3 ', '2 ', '2 ', '1 ', '3 ...","Your initial code serves as a starting point, ..."
1,Develop a Python program that takes the name o...,file_input = input() file_open = open(fil...,"def substitute_vowels(chunk, vowel_substitutes...",3 : Often,2 : Sometimes,1 : Almost Never,3 : Often,2 : Sometimes,2 : Sometimes,1 : Almost Never,...,3 : Often,3 : Often,2 : Sometimes,2 : Sometimes,2 : Sometimes,3 : Often,3 : Often,1 : Almost Never,"['3 ', '2 ', '1 ', '3 ', '2 ', '2 ', '1 ', '3 ...",Your code exhibits a solid attempt at reading ...
2,Develop a Python program that takes the name o...,file_input = input() file_open = open(fil...,"def substitute_vowels(chunk, vowel_substitutes...",3 : Often,2 : Sometimes,1 : Almost Never,3 : Often,2 : Sometimes,2 : Sometimes,1 : Almost Never,...,3 : Often,3 : Often,2 : Sometimes,2 : Sometimes,2 : Sometimes,3 : Often,3 : Often,1 : Almost Never,"['3 ', '2 ', '1 ', '3 ', '2 ', '2 ', '1 ', '3 ...",It looks like you're in a good place with some...
3,Develop a Python program that takes the name o...,file_input = input() file_open = open(fil...,"def substitute_vowels(chunk, vowel_substitutes...",3 : Often,2 : Sometimes,1 : Almost Never,3 : Often,2 : Sometimes,2 : Sometimes,1 : Almost Never,...,3 : Often,3 : Often,2 : Sometimes,2 : Sometimes,2 : Sometimes,3 : Often,3 : Often,1 : Almost Never,"['3 ', '2 ', '1 ', '3 ', '2 ', '2 ', '1 ', '3 ...",Your approach to reading the file and splittin...


In [16]:
dataset = df[5000:10001]

In [17]:
dataset.shape

(5001, 21)

In [ ]:
dataset.head(10)

,Problem,Student_code,Expected_code,Q01,Q02,Q03,Q04,Q05,Q06,Q07,...,Q09,Q10,Q11,Q12,Q13,Q14,Q15,Q16,metacognitive_vector,metacognitive_feedback
5000,Develop a Python program that takes an m x n m...,"file_name=input() file=open(file_name,'r'...",def setZeroes(input_filename): with o...,3 : Often,2 : Sometimes,3 : Often,2 : Sometimes,3 : Often,2 : Sometimes,3 : Often,...,2 : Sometimes,2 : Sometimes,2 : Sometimes,2 : Sometimes,3 : Often,3 : Often,1 : Almost Never,2 : Sometimes,"['3 ', '2 ', '3 ', '2 ', '3 ', '2 ', '3 ', '2 ...",Your approach to reading the input file and ex...
5001,Develop a Python program that takes an m x n m...,"file_name=input() file=open(file_name,'r'...",def setZeroes(input_filename): with o...,3 : Often,2 : Sometimes,3 : Often,2 : Sometimes,3 : Often,2 : Sometimes,3 : Often,...,2 : Sometimes,2 : Sometimes,2 : Sometimes,2 : Sometimes,3 : Often,3 : Often,1 : Almost Never,2 : Sometimes,"['3 ', '2 ', '3 ', '2 ', '3 ', '2 ', '3 ', '2 ...",Your approach to the problem offers a good sta...
5002,Develop a Python program that takes an m x n m...,"file_name=input() file=open(file_name,'r'...",def setZeroes(input_filename): with o...,3 : Often,2 : Sometimes,3 : Often,2 : Sometimes,3 : Often,2 : Sometimes,3 : Often,...,2 : Sometimes,2 : Sometimes,2 : Sometimes,2 : Sometimes,3 : Often,3 : Often,1 : Almost Never,2 : Sometimes,"['3 ', '2 ', '3 ', '2 ', '3 ', '2 ', '3 ', '2 ...",Your approach to reading the input file shows ...
5003,Develop a Python program that takes an m x n m...,"file_name=input() file=open(file_name,'r'...",def setZeroes(input_filename): with o...,3 : Often,2 : Sometimes,3 : Often,2 : Sometimes,3 : Often,2 : Sometimes,3 : Often,...,2 : Sometimes,2 : Sometimes,2 : Sometimes,2 : Sometimes,3 : Often,3 : Often,1 : Almost Never,2 : Sometimes,"['3 ', '2 ', '3 ', '2 ', '3 ', '2 ', '3 ', '2 ...",Your approach to the problem shows that you ha...
5004,Develop a Python program that takes an m x n m...,"file_name=input() file=open(file_name,'r'...",def setZeroes(input_filename): with o...,3 : Often,2 : Sometimes,3 : Often,2 : Sometimes,3 : Often,2 : Sometimes,3 : Often,...,2 : Sometimes,2 : Sometimes,2 : Sometimes,2 : Sometimes,3 : Often,3 : Often,1 : Almost Never,2 : Sometimes,"['3 ', '2 ', '3 ', '2 ', '3 ', '2 ', '3 ', '2 ...",Your approach to reading the input file and in...
5005,Develop a Python program that takes an m x n m...,#get the file name as input file_name=inp...,def setZeroes(input_filename): with o...,3 : Often,2 : Sometimes,3 : Often,2 : Sometimes,3 : Often,2 : Sometimes,3 : Often,...,2 : Sometimes,2 : Sometimes,2 : Sometimes,2 : Sometimes,3 : Often,3 : Often,1 : Almost Never,2 : Sometimes,"['3 ', '2 ', '3 ', '2 ', '3 ', '2 ', '3 ', '2 ...",Your approach to identifying the positions of ...
5006,Develop a Python program that takes an m x n m...,#get the file name as input file_name=inp...,def setZeroes(input_filename): with o...,3 : Often,2 : Sometimes,3 : Often,2 : Sometimes,3 : Often,2 : Sometimes,3 : Often,...,2 : Sometimes,2 : Sometimes,2 : Sometimes,2 : Sometimes,3 : Often,3 : Often,1 : Almost Never,2 : Sometimes,"['3 ', '2 ', '3 ', '2 ', '3 ', '2 ', '3 ', '2 ...",Your approach reflects a solid understanding o...
5007,Develop a Python program that takes an m x n m...,#get the file name as input file_name=inp...,def setZeroes(input_filename): with o...,3 : Often,2 : Sometimes,3 : Often,2 : Sometimes,3 : Often,2 : Sometimes,3 : Often,...,2 : Sometimes,2 : Sometimes,2 : Sometimes,2 : Sometimes,3 : Often,3 : Often,1 : Almost Never,2 : Sometimes,"['3 ', '2 ', '3 ', '2 ', '3 ', '2 ', '3 ', '2 ...",Your attempt at addressing the problem shows t...
5008,Develop a Python program that takes an m x n m...,#get the file name as input file_name=inp...,def setZeroes(input_filename): with o...,3 : Often,2 : Sometimes,3 : Often,2 : Sometimes,3 : Often,2 : Sometimes,3 : Often,...,2 : Sometimes,2 : Sometimes,2 : Sometimes,2 : Sometimes,3 : Often,3 : Often,1 : Almost Never,2 : Sometimes,"['3 ', '2 ', '3 ', '2 '

# Generating Metacognitive feedback

In [ ]:
dataset.tail()

,Problem,Student_code,Expected_code,Q01,Q02,Q03,Q04,Q05,Q06,Q07,...,Q09,Q10,Q11,Q12,Q13,Q14,Q15,Q16,metacognitive_vector,metacognitive_feedback
9996,Amicable numbers are pairs of numbers for whic...,def named_are_amicable(): for n...,def sum_of_divisors(number): divisors...,2 : Sometimes,2 : Sometimes,2 : Sometimes,3 : Often,1 : Almost Never,2 : Sometimes,2 : Sometimes,...,3 : Often,2 : Sometimes,2 : Sometimes,2 : Sometimes,1 : Almost Never,2 : Sometimes,3 : Often,2 : Sometimes,"['2 ', '2 ', '2 ', '3 ', '1 ', '2 ', '2 ', '2 ...",Your current implementation has several issues...
9997,Amicable numbers are pairs of numbers for whic...,def named_are_amicable(): for n...,def sum_of_divisors(number): divisors...,2 : Sometimes,2 : Sometimes,2 : Sometimes,3 : Often,1 : Almost Never,2 : Sometimes,2 : Sometimes,...,3 : Often,2 : Sometimes,2 : Sometimes,2 : Sometimes,1 : Almost Never,2 : Sometimes,3 : Often,2 : Sometimes,"['2 ', '2 ', '2 ', '3 ', '1 ', '2 ', '2 ', '2 ...",Your approach to coding the solution shows pot...
9998,Amicable numbers are pairs of numbers for whic...,def named_are_amicable(): for n...,def sum_of_divisors(number): divisors...,2 : Sometimes,2 : Sometimes,2 : Sometimes,3 : Often,1 : Almost Never,2 : Sometimes,2 : Sometimes,...,3 : Often,2 : Sometimes,2 : Sometimes,2 : Sometimes,1 : Almost Never,2 : Sometimes,3 : Often,2 : Sometimes,"['2 ', '2 ', '2 ', '3 ', '1 ', '2 ', '2 ', '2 ...","Your approach to the problem is a good start, ..."
9999,Amicable numbers are pairs of numbers for whic...,def named_are_amicable(): for n...,def sum_of_divisors(number): divisors...,2 : Sometimes,2 : Sometimes,2 : Sometimes,3 : Often,1 : Almost Never,2 : Sometimes,2 : Sometimes,...,3 : Often,2 : Sometimes,2 : Sometimes,2 : Sometimes,1 : Almost Never,2 : Sometimes,3 : Often,2 : Sometimes,"['2 ', '2 ', '2 ', '3 ', '1 ', '2 ', '2 ', '2 ...",Your approach shows some understanding of loop...
10000,Amicable numbers are pairs of numbers for whic...,def named_are_amicable(): for n...,def sum_of_divisors(number): divisors...,2 : Sometimes,2 : Sometimes,2 : Sometimes,3 : Often,1 : Almost Never,2 : Sometimes,2 : Sometimes,...,3 : Often,2 : Sometimes,2 : Sometimes,2 : Sometimes,1 : Almost Never,2 : Sometimes,3 : Often,2 : Sometimes,"['2 ', '2 ', '2 ', '3 ', '1 ', '2 ', '2 ', '2 ...",Your implementation has a few significant issu...


In [18]:
data1 = dataset[:1000]

In [19]:
data1.tail()

,Problem,Student_code,Expected_code,Q01,Q02,Q03,Q04,Q05,Q06,Q07,...,Q09,Q10,Q11,Q12,Q13,Q14,Q15,Q16,metacognitive_vector,metacognitive_feedback
5995,PROBLEM: Develop a Python program that ta...,input_file = input() #getting input readi...,def update_inventory(data): inventory...,2 : Sometimes,3 : Often,3 : Often,2 : Sometimes,3 : Often,3 : Often,1 : Almost Never,...,3 : Often,2 : Sometimes,3 : Often,2 : Sometimes,2 : Sometimes,3 : Often,2 : Sometimes,3 : Often,"['2 ', '3 ', '3 ', '2 ', '3 ', '3 ', '1 ', '2 ...",Your approach to solving the problem shows pro...
5996,PROBLEM: Develop a Python program that ta...,input_file = input() #getting input readi...,def update_inventory(data): inventory...,2 : Sometimes,3 : Often,3 : Often,2 : Sometimes,3 : Often,3 : Often,1 : Almost Never,...,3 : Often,2 : Sometimes,3 : Often,2 : Sometimes,2 : Sometimes,3 : Often,2 : Sometimes,3 : Often,"['2 ', '3 ', '3 ', '2 ', '3 ', '3 ', '1 ', '2 ...",Your approach to handling the input file is a ...
5997,PROBLEM: Develop a Python program that ta...,input_file = input() #getting input readi...,def update_inventory(data): inventory...,2 : Sometimes,3 : Often,3 : Often,2 : Sometimes,3 : Often,3 : Often,1 : Almost Never,...,3 : Often,2 : Sometimes,3 : Often,2 : Sometimes,2 : Sometimes,3 : Often,2 : Sometimes,3 : Often,"['2 ', '3 ', '3 ', '2 ', '3 ', '3 ', '1 ', '2 ...",Your approach of reading the input file and ma...
5998,PROBLEM: Develop a Python program that ta...,input_file = input() #getting input readi...,def update_inventory(data): inventory...,2 : Sometimes,3 : Often,3 : Often,2 : Sometimes,3 : Often,3 : Often,1 : Almost Never,...,3 : Often,2 : Sometimes,3 : Often,2 : Sometimes,2 : Sometimes,3 : Often,2 : Sometimes,3 : Often,"['2 ', '3 ', '3 ', '2 ', '3 ', '3 ', '1 ', '2 ...",Your approach to solving the inventory problem...
5999,PROBLEM: Develop a Python program that ta...,"def available_stock(product_list,stock_list): ...",def update_inventory(data): inventory...,3 : Often,1 : Almost Never,1 : Almost Never,3 : Often,2 : Sometimes,3 : Often,1 : Almost Never,...,3 : Often,3 : Often,3 : Often,2 : Sometimes,2 : Sometimes,3 : Often,2 : Sometimes,1 : Almost Never,"['3 ', '1 ', '1 ', '3 ', '2 ', '3 ', '1 ', '2 ...",Your approach signals that you're often able t...


In [20]:
data2 = dataset[1000:2000]

In [22]:
data2.head()

,Problem,Student_code,Expected_code,Q01,Q02,Q03,Q04,Q05,Q06,Q07,...,Q09,Q10,Q11,Q12,Q13,Q14,Q15,Q16,metacognitive_vector,metacognitive_feedback
6000,PROBLEM: Develop a Python program that ta...,"def available_stock(product_list,stock_list): ...",def update_inventory(data): inventory...,3 : Often,1 : Almost Never,1 : Almost Never,3 : Often,2 : Sometimes,3 : Often,1 : Almost Never,...,3 : Often,3 : Often,3 : Often,2 : Sometimes,2 : Sometimes,3 : Often,2 : Sometimes,1 : Almost Never,"['3 ', '1 ', '1 ', '3 ', '2 ', '3 ', '1 ', '2 ...",Your approach to solving the problem shows som...
6001,PROBLEM: Develop a Python program that ta...,"def available_stock(product_list,stock_list): ...",def update_inventory(data): inventory...,3 : Often,1 : Almost Never,1 : Almost Never,3 : Often,2 : Sometimes,3 : Often,1 : Almost Never,...,3 : Often,3 : Often,3 : Often,2 : Sometimes,2 : Sometimes,3 : Often,2 : Sometimes,1 : Almost Never,"['3 ', '1 ', '1 ', '3 ', '2 ', '3 ', '1 ', '2 ...",Your approach to solving the problem shows som...
6002,PROBLEM: Develop a Python program that ta...,"def available_stock(product_list,stock_list): ...",def update_inventory(data): inventory...,3 : Often,1 : Almost Never,1 : Almost Never,3 : Often,2 : Sometimes,3 : Often,1 : Almost Never,...,3 : Often,3 : Often,3 : Often,2 : Sometimes,2 : Sometimes,3 : Often,2 : Sometimes,1 : Almost Never,"['3 ', '1 ', '1 ', '3 ', '2 ', '3 ', '1 ', '2 ...","Your initial approach shows promise, but there..."
6003,PROBLEM: Develop a Python program that ta...,"def available_stock(product_list,stock_list): ...",def update_inventory(data): inventory...,3 : Often,1 : Almost Never,1 : Almost Never,3 : Often,2 : Sometimes,3 : Often,1 : Almost Never,...,3 : Often,3 : Often,3 : Often,2 : Sometimes,2 : Sometimes,3 : Often,2 : Sometimes,1 : Almost Never,"['3 ', '1 ', '1 ', '3 ', '2 ', '3 ', '1 ', '2 ...",Your approach to structuring the solution usin...
6004,PROBLEM: Develop a Python program that ta...,"def available_stock(product_list,stock_list): ...",def update_inventory(data): inventory...,3 : Often,1 : Almost Never,1 : Almost Never,3 : Often,2 : Sometimes,3 : Often,1 : Almost Never,...,3 : Often,3 : Often,3 : Often,2 : Sometimes,2 : Sometimes,3 : Often,2 : Sometimes,1 : Almost Never,"['3 ', '1 ', '1 ', '3 ', '2 ', '3 ', '1 ', '2 ...",Your approach shows a good understanding of th...


In [ ]:
data3 = dataset[2000:3000]

In [ ]:
data4 = dataset[3000:4000]

In [37]:
data5 = dataset[4000:]

In [ ]:
data1.head(3)

,Problem,Student_code,Expected_code,Q01,Q02,Q03,Q04,Q05,Q06,Q07,...,Q09,Q10,Q11,Q12,Q13,Q14,Q15,Q16,metacognitive_vector,metacognitive_feedback
5000,Develop a Python program that takes an m x n m...,"file_name=input() file=open(file_name,'r'...",def setZeroes(input_filename): with o...,3 : Often,2 : Sometimes,3 : Often,2 : Sometimes,3 : Often,2 : Sometimes,3 : Often,...,2 : Sometimes,2 : Sometimes,2 : Sometimes,2 : Sometimes,3 : Often,3 : Often,1 : Almost Never,2 : Sometimes,"['3 ', '2 ', '3 ', '2 ', '3 ', '2 ', '3 ', '2 ...",Your approach to reading the input file and ex...
5001,Develop a Python program that takes an m x n m...,"file_name=input() file=open(file_name,'r'...",def setZeroes(input_filename): with o...,3 : Often,2 : Sometimes,3 : Often,2 : Sometimes,3 : Often,2 : Sometimes,3 : Often,...,2 : Sometimes,2 : Sometimes,2 : Sometimes,2 : Sometimes,3 : Often,3 : Often,1 : Almost Never,2 : Sometimes,"['3 ', '2 ', '3 ', '2 ', '3 ', '2 ', '3 ', '2 ...",Your approach to the problem offers a good sta...
5002,Develop a Python program that takes an m x n m...,"file_name=input() file=open(file_name,'r'...",def setZeroes(input_filename): with o...,3 : Often,2 : Sometimes,3 : Often,2 : Sometimes,3 : Often,2 : Sometimes,3 : Often,...,2 : Sometimes,2 : Sometimes,2 : Sometimes,2 : Sometimes,3 : Often,3 : Often,1 : Almost Never,2 : Sometimes,"['3 ', '2 ', '3 ', '2 ', '3 ', '2 ', '3 ', '2 ...",Your approach to reading the input file shows ...


In [ ]:
data5.tail()

,Problem,Student_code,Expected_code,Q01,Q02,Q03,Q04,Q05,Q06,Q07,...,Q09,Q10,Q11,Q12,Q13,Q14,Q15,Q16,metacognitive_vector,metacognitive_feedback
9996,Amicable numbers are pairs of numbers for whic...,def named_are_amicable(): for n...,def sum_of_divisors(number): divisors...,2 : Sometimes,2 : Sometimes,2 : Sometimes,3 : Often,1 : Almost Never,2 : Sometimes,2 : Sometimes,...,3 : Often,2 : Sometimes,2 : Sometimes,2 : Sometimes,1 : Almost Never,2 : Sometimes,3 : Often,2 : Sometimes,"['2 ', '2 ', '2 ', '3 ', '1 ', '2 ', '2 ', '2 ...",Your current implementation has several issues...
9997,Amicable numbers are pairs of numbers for whic...,def named_are_amicable(): for n...,def sum_of_divisors(number): divisors...,2 : Sometimes,2 : Sometimes,2 : Sometimes,3 : Often,1 : Almost Never,2 : Sometimes,2 : Sometimes,...,3 : Often,2 : Sometimes,2 : Sometimes,2 : Sometimes,1 : Almost Never,2 : Sometimes,3 : Often,2 : Sometimes,"['2 ', '2 ', '2 ', '3 ', '1 ', '2 ', '2 ', '2 ...",Your approach to coding the solution shows pot...
9998,Amicable numbers are pairs of numbers for whic...,def named_are_amicable(): for n...,def sum_of_divisors(number): divisors...,2 : Sometimes,2 : Sometimes,2 : Sometimes,3 : Often,1 : Almost Never,2 : Sometimes,2 : Sometimes,...,3 : Often,2 : Sometimes,2 : Sometimes,2 : Sometimes,1 : Almost Never,2 : Sometimes,3 : Often,2 : Sometimes,"['2 ', '2 ', '2 ', '3 ', '1 ', '2 ', '2 ', '2 ...","Your approach to the problem is a good start, ..."
9999,Amicable numbers are pairs of numbers for whic...,def named_are_amicable(): for n...,def sum_of_divisors(number): divisors...,2 : Sometimes,2 : Sometimes,2 : Sometimes,3 : Often,1 : Almost Never,2 : Sometimes,2 : Sometimes,...,3 : Often,2 : Sometimes,2 : Sometimes,2 : Sometimes,1 : Almost Never,2 : Sometimes,3 : Often,2 : Sometimes,"['2 ', '2 ', '2 ', '3 ', '1 ', '2 ', '2 ', '2 ...",Your approach shows some understanding of loop...
10000,Amicable numbers are pairs of numbers for whic...,def named_are_amicable(): for n...,def sum_of_divisors(number): divisors...,2 : Sometimes,2 : Sometimes,2 : Sometimes,3 : Often,1 : Almost Never,2 : Sometimes,2 : Sometimes,...,3 : Often,2 : Sometimes,2 : Sometimes,2 : Sometimes,1 : Almost Never,2 : Sometimes,3 : Often,2 : Sometimes,"['2 ', '2 ', '2 ', '3 ', '1 ', '2 ', '2 ', '2 ...",Your implementation has a few significant issu...


In [30]:
def generate_metacognitive_feedback(row):
    # Extract details from the row
    problem = row['Problem']
    student_answer = row['Student_code']
    expected_answer = row['Expected_code']
    metacognitive_vector = row['metacognitive_vector']

    # Format the prompt using LangChain's PromptTemplate
    prompt = prompt_template.format(
        problem=problem,
        answer=student_answer,
        solution=expected_answer,
        metacognitive_vector=metacognitive_vector
    )

    # Run the Mistral model to get feedback

    try:
        feedback = generate_response(prompt)
        # print(feedback)
        print(f"Feedback generated for row {row.name}")
    except Exception as e:
        feedback = "None"
        print(e)

    return feedback


In [32]:
# Assuming `generate_metacognitive_feedback` is a function that takes a row as input
# for index, row in df.iterrows():
    # data1.loc[index, 'mistral_metacognitive_feedback'] = generate_metacognitive_feedback(row)
data2['mistral_metacognitive_feedback'] = data2.apply(generate_metacognitive_feedback, axis=1)

Feedback generated for row 6000
Feedback generated for row 6001
Feedback generated for row 6002
Feedback generated for row 6003
Feedback generated for row 6004
Feedback generated for row 6005
Feedback generated for row 6006
Feedback generated for row 6007
Feedback generated for row 6008
Feedback generated for row 6009
Feedback generated for row 6010
Feedback generated for row 6011
Feedback generated for row 6012
Feedback generated for row 6013
Feedback generated for row 6014
Feedback generated for row 6015
Feedback generated for row 6016
Feedback generated for row 6017
Feedback generated for row 6018
Feedback generated for row 6019
Feedback generated for row 6020
Feedback generated for row 6021
Feedback generated for row 6022
Feedback generated for row 6023
Feedback generated for row 6024
Feedback generated for row 6025
Feedback generated for row 6026
Feedback generated for row 6027
Feedback generated for row 6028
Feedback generated for row 6029
Feedback generated for row 6030
Feedback

<ipython-input-32-9ee971183632>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['mistral_metacognitive_feedback'] = data2.apply(generate_metacognitive_feedback, axis=1)


In [34]:
# Save the enhanced data with feedback to a CSV file
output_file_path = "final_dataset_with_annotated_metacognitive_feedback_gpt-4o-mini_and_mistral_6000_6999.csv"
data2.to_csv(output_file_path, index=False)

In [38]:
data5.head(3)

,Problem,Student_code,Expected_code,Q01,Q02,Q03,Q04,Q05,Q06,Q07,...,Q09,Q10,Q11,Q12,Q13,Q14,Q15,Q16,metacognitive_vector,metacognitive_feedback
9000,PROBLEM: You are managing a cinema that h...,def take_input(file_name): with open(...,def get_row_with_max_free_seats(rows): ...,3 : Often,2 : Sometimes,1 : Almost Never,2 : Sometimes,3 : Often,2 : Sometimes,2 : Sometimes,...,3 : Often,3 : Often,2 : Sometimes,3 : Often,1 : Almost Never,3 : Often,2 : Sometimes,2 : Sometimes,"['3 ', '2 ', '1 ', '2 ', '3 ', '2 ', '2 ', '3 ...",Your approach to the problem demonstrates a go...
9001,PROBLEM: You are managing a cinema that h...,def take_input(file_name): with open(...,def get_row_with_max_free_seats(rows): ...,3 : Often,2 : Sometimes,1 : Almost Never,2 : Sometimes,3 : Often,2 : Sometimes,2 : Sometimes,...,3 : Often,3 : Often,2 : Sometimes,3 : Often,1 : Almost Never,3 : Often,2 : Sometimes,2 : Sometimes,"['3 ', '2 ', '1 ', '2 ', '3 ', '2 ', '2 ', '3 ...",Your approach to reading the file and splittin...
9002,PROBLEM: You are managing a cinema that h...,def take_input(file_name): with open(...,def get_row_with_max_free_seats(rows): ...,3 : Often,2 : Sometimes,1 : Almost Never,2 : Sometimes,3 : Often,2 : Sometimes,2 : Sometimes,...,3 : Often,3 : Often,2 : Sometimes,3 : Often,1 : Almost Never,3 : Often,2 : Sometimes,2 : Sometimes,"['3 ', '2 ', '1 ', '2 ', '3 ', '2 ', '2 ', '3 ...",Your approach of defining a function to read t...


In [39]:
data5.tail()

,Problem,Student_code,Expected_code,Q01,Q02,Q03,Q04,Q05,Q06,Q07,...,Q09,Q10,Q11,Q12,Q13,Q14,Q15,Q16,metacognitive_vector,metacognitive_feedback
9996,Amicable numbers are pairs of numbers for whic...,def named_are_amicable(): for n...,def sum_of_divisors(number): divisors...,2 : Sometimes,2 : Sometimes,2 : Sometimes,3 : Often,1 : Almost Never,2 : Sometimes,2 : Sometimes,...,3 : Often,2 : Sometimes,2 : Sometimes,2 : Sometimes,1 : Almost Never,2 : Sometimes,3 : Often,2 : Sometimes,"['2 ', '2 ', '2 ', '3 ', '1 ', '2 ', '2 ', '2 ...",Your current implementation has several issues...
9997,Amicable numbers are pairs of numbers for whic...,def named_are_amicable(): for n...,def sum_of_divisors(number): divisors...,2 : Sometimes,2 : Sometimes,2 : Sometimes,3 : Often,1 : Almost Never,2 : Sometimes,2 : Sometimes,...,3 : Often,2 : Sometimes,2 : Sometimes,2 : Sometimes,1 : Almost Never,2 : Sometimes,3 : Often,2 : Sometimes,"['2 ', '2 ', '2 ', '3 ', '1 ', '2 ', '2 ', '2 ...",Your approach to coding the solution shows pot...
9998,Amicable numbers are pairs of numbers for whic...,def named_are_amicable(): for n...,def sum_of_divisors(number): divisors...,2 : Sometimes,2 : Sometimes,2 : Sometimes,3 : Often,1 : Almost Never,2 : Sometimes,2 : Sometimes,...,3 : Often,2 : Sometimes,2 : Sometimes,2 : Sometimes,1 : Almost Never,2 : Sometimes,3 : Often,2 : Sometimes,"['2 ', '2 ', '2 ', '3 ', '1 ', '2 ', '2 ', '2 ...","Your approach to the problem is a good start, ..."
9999,Amicable numbers are pairs of numbers for whic...,def named_are_amicable(): for n...,def sum_of_divisors(number): divisors...,2 : Sometimes,2 : Sometimes,2 : Sometimes,3 : Often,1 : Almost Never,2 : Sometimes,2 : Sometimes,...,3 : Often,2 : Sometimes,2 : Sometimes,2 : Sometimes,1 : Almost Never,2 : Sometimes,3 : Often,2 : Sometimes,"['2 ', '2 ', '2 ', '3 ', '1 ', '2 ', '2 ', '2 ...",Your approach shows some understanding of loop...
10000,Amicable numbers are pairs of numbers for whic...,def named_are_amicable(): for n...,def sum_of_divisors(number): divisors...,2 : Sometimes,2 : Sometimes,2 : Sometimes,3 : Often,1 : Almost Never,2 : Sometimes,2 : Sometimes,...,3 : Often,2 : Sometimes,2 : Sometimes,2 : Sometimes,1 : Almost Never,2 : Sometimes,3 : Often,2 : Sometimes,"['2 ', '2 ', '2 ', '3 ', '1 ', '2 ', '2 ', '2 ...",Your implementation has a few significant issu...


In [ ]:
data5['mistral_metacognitive_feedback'] = data5.apply(generate_metacognitive_feedback, axis=1)

Feedback generated for row 9000
Feedback generated for row 9001
Feedback generated for row 9002
Feedback generated for row 9003
Feedback generated for row 9004
Feedback generated for row 9005
Feedback generated for row 9006
Feedback generated for row 9007
Feedback generated for row 9008
Feedback generated for row 9009
Feedback generated for row 9010
Feedback generated for row 9011
Feedback generated for row 9012
Feedback generated for row 9013
Feedback generated for row 9014
Feedback generated for row 9015
Feedback generated for row 9016
Feedback generated for row 9017
Feedback generated for row 9018
Feedback generated for row 9019
Feedback generated for row 9020
Feedback generated for row 9021
Feedback generated for row 9022
Feedback generated for row 9023
Feedback generated for row 9024
Feedback generated for row 9025
Feedback generated for row 9026
Feedback generated for row 9027
Feedback generated for row 9028
Feedback generated for row 9029
Feedback generated for row 9030
Feedback

In [ ]:
# Save the enhanced data with feedback to a CSV file
output_file_path = "final_dataset_with_annotated_metacognitive_feedback_gpt-4o-mini_and_mistral_9000_10000.csv"
data5.to_csv(output_file_path, index=False)

In [35]:
data2.iloc[500]['mistral_metacognitive_feedback']

"Your approach to reading the file and processing the input is a good start, but it's clear that you need to break down the problem into smaller, manageable tasks to fully address the requirements. Start by ensuring you understand the key components of a magic square: unique numbers within a specific range and equal sums for rows, columns, and diagonals. You've begun by reading the file, but you need to transform this input into a usable matrix format. Think about how you can convert the list of strings into a list of lists of integers.\n\nNext, consider the validation steps. You need to check if all numbers in the matrix are unique and within the valid range. Sketch out a plan for how you can verify these conditions before moving on to the sum checks. Remember to systematically verify each part of your algorithm to ensure correctness. For the sum checks, think about how you can iterate through the matrix to calculate the sums of rows, columns, and diagonals. It might help to manually 

In [ ]:
data5.iloc[500]['mistral_metacognitive_feedback']